## Smile Recognition Using Haar Cascades

### Semester 5: Computer Vision LEC - LA01

#### Cindy Amanda Onggirawan - 2440055351 || Jocelyn Michelle - 2440039796

Perkembangan teknologi zaman sekarang terus meningkat. Tidak hanya itu, kebutuhan pengguna juga terus berkembang dari yang primer dan mulai memasuki hal-hal sekunder, salah satunya adalah melalui pendeteksi wajah dan ekspresi wajah, seperti senyum (smile). Disini kita hendak mengembangkan sebuah smile detection system yang diimplementasikan melalui Python. Perkembangan selanjutnya bisa dilakukan dengan meng-enhance input yang diterima agar dapat melakukan prediksi dengan lebih baik. 

Langkah pertama pengerjaan ialah meng-import library yang akan digunakan. Disini, kita memakai 3 library: 
- CV2 (Untuk pengerjaan dengan gambar)
- OS (Untuk pengerjaan dengan directory files)
- Numpy (Untuk pengerjaan dengan array)

In [1]:
import cv2 
import os
import numpy as np

Disini, kita akan menggunakan model face detector yang telah disediakan oleh OpenCV Haar Cascade, yaitu untuk Face Recognizer dan Smile Recognizer. Maka dari itu, kita akan menggunakan kedua cascade tersebut. 

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

Berikutnya, kita akan melakukan pembacaan dataset train, yang didalamnya telah terbagi ke dalam dua folder: Smile dan No Smile. Disini kita melakukan face recognition menggunakan cascade yang sebelumnya telah kita import (Face Cascde), yang returnnya berupa jumlah wajah yang terdeteksi pada gambar. 
Untuk memproses gambar yang akan digunakan, pertama-tama kita melakukan resize pada gambar dikarenakan ukurannya yang cukup besar, sehingga akan cukup berat apabila diimplementasikan untuk dataset dengan jumlah data yang cukup besar. Selanjutnya, kita perlu mengkonversi gambar ke dalam versi grayscaled-nya, sebab model Haar Cascade akan diaplikasikan pada gambar hitam-putih (grayscaled images). 

Untuk hasilnya akan kita simpan dalam variable train_images dan indexnya dalam variable train_images_idx. 

In [3]:
train_path = 'dataset/train'
train_images_type = os.listdir(train_path)

train_images = []
train_images_idx = []

for index, face_type in enumerate(train_images_type):
    img_path = f"{train_path}/{face_type}"
    
    for imgs in os.listdir(img_path):
        full_path = f"{img_path}/{imgs}"
        img = cv2.imread(full_path)
        width = int(img.shape[1] * 0.2)
        height = int(img.shape[0] * 0.2)
        img = cv2.resize(img, (width, height))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        face_rect = face_cascade.detectMultiScale(gray, 1.3, 3)
        if (len(face_rect) < 1): continue 

        for (x, y, w, h) in face_rect:
            face = gray[y:y+h, w:w+h]
            train_images.append(face)
            train_images_idx.append(index)

Selanjutnya, dataset yang telah kita train sebelumnya akan kita feed pada LBPH (Local Binary Pattern Histograms) Face Recognizer. LBPH ini kita gunakan untuk merecognize wajah yang terdapat dalam dataset ini tergolong ke dalam tipe yang mana, _smile_ atau _no smile_. 

Cara kerjanya pun dapat dibilang cukup sederhana. LBPH merupakan penggabungan dari LBP (Local Binary Pattern) dan descriptors dari HOG (Histograms of Oriented Gradients). Pada kasus ini, kita hanya akan mengoperkan 2 parameter yaitu radius (jarak di sekitar pixel tengah) dan neighbors (jumlah pixel yang digunakan pada sekitar center pixel).

In [5]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(train_images, np.array(train_images_idx))

Sekarang kita dapat menggunakan model yang telah dipersiapkan sebelumnya untuk test dataset kita. Langkah-langkah yang dilakukan sama seperti yang kita sebelumnya lakukan untuk training dataset, namun disini kita menambahkan satu recognizer yaitu *smile detector* untuk mendeteksi keberadaan ekspresi senyum pada wajah, yang juga akan mengimplementasikan model Haar cascade. Hasil akan ditunjukkan dengan mengkotakkan wajah dan menunjukkan apakah ekspresi wajah tersebut sedang tersenyum atau tidak. 

Disini, kita membuat model untuk mendeteksi hanya satu wajah manusia, dengan data yang diambil dari local yang sudah disediakan. Kedepannya kita dapat meng-improve model ini dengan memperbolehkan pendeteksian beberapa wajah sekaligus, dengan data yang didapatkan dari user input maupun diambil secara real-time menggunakan kamera, serta dapat mendeteksi berbagai ekspresi wajah lainnya juga. 

In [17]:
test_path = 'dataset/test'

for images in os.listdir(test_path):
    full_image_path = f"{test_path}/{images}"
    img = cv2.imread(full_image_path)
    # width = int(img.shape[1] * 0.08)
    # height = int(img.shape[0] * 0.08)
    img = cv2.resize(img, (width, height))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    detected_faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    print(f"{len(detected_faces)} detected_faces")

    for (x, y, w, h) in detected_faces:
        if (len(detected_faces) < 1): 
            # no faces detected
            cv2.putText(img, "No Face Found", (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0,0,255), 3)

        else:
            face_img = gray[y:y+h, x:x+w]
            smile_rect, reject_level, level_weight = smile_cascade.detectMultiScale3(face_img, 2.5, 20, outputRejectLevels=True)

            result, confidence = face_recognizer.predict(face_img)
            
            text = f"{[result]} - {confidence}"
            cv2.putText(img, text, (x, y+h+15), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,255,0))

            if len(level_weight) == 0:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
                cv2.putText(img, "Not Smiling", (x+3, y+h-5), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,0,255), 1)
            else:
                if max(level_weight) < 0:
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
                    cv2.putText(img, "Not Smiling", (x+3, y+h-5), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,0,255), 1)
                else:
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0,255,0), 3)
                    cv2.putText(img, "Smiling", (x+3, y+h-5), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,255,0), 1)

    cv2.imshow(f"Image {images}", img)
    cv2.waitKey(0)

cv2.destroyAllWindows()
cv2.waitKey(0)

1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces


-1

Model ini mendeteksi satu wajah untuk setiap test data yang sudah disediakan. Hasil test menunjukkan bahwa 9 dari 10 gambar terdeteksi dengan benar (smiling face), sedangkan 1 gambar (kiri, [3]) yang menunjukkan ekspresi sedih  namun terdeteksi sedang tersenyum. Ini menandakan bahwa model kita masih harus diperbaiki lagi.